In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import os
import pandas as pd
from requests import request
from tqdm import tqdm

import property_scraper
from property_scraper.scraper import Scraper
#from property_scraper.pvp_scraper import PVPScraper

In [3]:
scraper = Scraper()
#pvp = PVPScraper()

In [4]:
filename = '../data/immobiliare_palermo.xlsx'
properties = pd.read_excel(filename, sheet_name='vendita-case')

In [5]:
print(len(properties))
print(len(properties[properties['Status'] == 'Not available']))

1022
1022


In [6]:
display(properties.head(3).T)

,0,1,2
Unnamed: 0.1,0,1,2
Unnamed: 0,0,1,2
index,0,1,2
nd-mediaObject__contentBottom_text,Visitamessaggio,Visitamessaggio,Visitamessaggio
in-photo_text,20sky,44sky,22sky
in-realEstateBadge_text,sky,sky,sky
in-realEstateListCard__actions_text,Visitamessaggio,Visitamessaggio,Visitamessaggio
in-card_text,"20skyQuadrilocale via Tommaso Marcellini 5, Ca...","44skyAppartamento via Case Spuches 9, Santicel...","22skyAppartamento largo Francesco Garufi 18, C..."
in-card_href,https://www.immobiliare.it/annunci/99876618/,https://www.immobiliare.it/annunci/101505517/,https://www.immobiliare.it/annunci/99875054/
nd-mediaObject--colToRow_text,"20skyQuadrilocale via Tommaso Marcellini 5, Ca...","44skyAppartamento via Case Spuches 9, Santicel...","22skyAppartamento largo Francesco Garufi 18, C..."


get your subscription key at https://rapidapi.com/restyler/api/scrapeninja from "Code generator"

copy and paste it to 'x-rapidapi-key' header below

```python
headers = {
    "Content-Type": "application/json",
    "x-rapidapi-host": "scrapeninja.p.rapidapi.com",
    "x-rapidapi-key": "YOUR-RAPIDAPI-KEY"
}
```

```python
```

In [7]:
!cp ../data/*.json /home/emanuele/.conda/envs/py310/lib/python3.10/site-packages/property_scraper/data/

In [8]:
#search_filename = os.path.abspath(os.path.join(os.path.dirname(property_scraper.__file__), 'data', 'pvp_scrapeninja.json'))


In [9]:
#properties['Status'] = 'Not available'
#properties['Content'] = None

In [10]:
urls = properties['Hyperlink'].unique()
urls[:10]

array(['https://www.immobiliare.it/annunci/99876618/',
       'https://www.immobiliare.it/annunci/101505517/',
       'https://www.immobiliare.it/annunci/99875054/',
       'https://www.immobiliare.it/annunci/97069048/',
       'https://www.immobiliare.it/annunci/93812752/',
       'https://www.immobiliare.it/annunci/101202243/',
       'https://www.immobiliare.it/annunci/101088063/',
       'https://www.immobiliare.it/annunci/101207693/',
       'https://www.immobiliare.it/annunci/99066666/',
       'https://www.immobiliare.it/annunci/90524107/'], dtype=object)

In [11]:
search_filename = os.path.abspath('../data/immobiliare_scrapeninja.json')
print(search_filename)
if os.path.exists(search_filename): 
    scraper.headers, scraper.payload = scraper.init_scrapeninja(search_filename)

/home/git/property_scraper/data/immobiliare_scrapeninja.json


ideas for improving performance:
1. use session;
2. use aiohttp;

    

In [12]:
%%time 

pbar = tqdm(properties[properties['Status'] == 'Not available'].index)
BATCH_SIZE = 10
#MAX_URLS = 3
ix = 0
for idx in pbar:
    url = properties.loc[idx, 'Hyperlink']
    #print(url)
    pbar.set_description(f"{url}")
    scraper.payload['url'] = url
    try:
        scraper.request = scraper.get_request_via_scrapeninja(scraper.headers, scraper.payload)
    except:
        scraper.request = None
    print(type(scraper.request))
    if scraper.request:
        if scraper.request.status_code == 200:
            properties.loc[idx, 'Status'] = 'Downloaded'
            properties.loc[idx, 'Content'] = scraper.request.content
            ix += 1
        else:
            print(f"Request for page {url} return status code {scraper.request.status_code}!")
    #if ix > MAX_URLS:
        #break
    if ix % BATCH_SIZE == 0: 
        properties.to_excel(filename, sheet_name='vendita-case')
properties.to_excel(filename, sheet_name='vendita-case')

nan: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1022/1022 [1:07:11<00:00,  3.94s/it]


In [13]:
#pvp.request = pvp.get_request_via_scrapeninja(scraper.headers, scraper.payload)
#properties.to_excel(filename, sheet_name='vendita-case')

In [14]:
display(properties[properties['Status'] == 'Downloaded'].head().T)

,0,1,2,3,4
Unnamed: 0.1,0,1,2,3,4
Unnamed: 0,0,1,2,3,4
index,0,1,2,3,4
nd-mediaObject__contentBottom_text,Visitamessaggio,Visitamessaggio,Visitamessaggio,Visitamessaggio,Visitamessaggio
in-photo_text,20sky,44sky,22sky,16sky,20vetrina
in-realEstateBadge_text,sky,sky,sky,sky,vetrina
in-realEstateListCard__actions_text,Visitamessaggio,Visitamessaggio,Visitamessaggio,Visitamessaggio,Visitamessaggio
in-card_text,"20skyQuadrilocale via Tommaso Marcellini 5, Ca...","44skyAppartamento via Case Spuches 9, Santicel...","22skyAppartamento largo Francesco Garufi 18, C...","16skyQuadrilocale via Gioiamia 12, Tribunale, ...","20vetrinaQuadrilocale via Carlo Goldoni, Malas..."
in-card_href,https://www.immobiliare.it/annunci/99876618/,https://www.immobiliare.it/annunci/101505517/,https://www.immobiliare.it/annunci/99875054/,https://www.immobiliare.it/annunci/97069048/,https://www.immobiliare.it/annunci/93812752/
nd-mediaObject--colToRow_text,"20skyQuadrilocale via Tommaso Marcellini 5, Ca...","44skyAppartamento via Case Spuches 9, Santicel...","22skyAppartamento largo Francesco Garufi 18, C...","16skyQuadrilocale via Gioiamia 12, Tribunale, ...","20vetrinaQuadrilocale via Carlo Goldoni, Malas..."


In [15]:
stop here

SyntaxError: invalid syntax (4067800170.py, line 1)

In [ ]:
pvp.request.status_code

In [ ]:
pvp.request.content.decode('utf-8')[:100]

In [ ]:
#
response_json = json.loads(pvp.request.text)
#print(pvp.request.__dict__)
print(json.dumps(response_json, indent=4))
#response_json

In [ ]:
results = pvp.get_results()
display(results.head())

In [ ]:
def get_all_responses():
    
    url = 'https://scrapeninja.p.rapidapi.com/scrape'
    response = request("POST", url, json=payload, headers=headers)
    ofilename = 'data/pvp.search_response.html'
    with open(ofilename, 'w') as f:
        f.write(response.text)
        

In [ ]:
#print(response.__dict__)

In [ ]:
response = requests.request("POST", url, json=payload, headers=headers)

